In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import monai
import torch
import numpy as np
import pandas as pd
import tqdm as tqdm
from monai.data import decollate_batch
from torch.utils.data import DataLoader
from monai.metrics import ROCAUCMetric
from monai.utils import set_determinism
import matplotlib.pyplot as plt
import PIL
from monai.transforms import LoadImage, Resize, NormalizeIntensity,RandRotate,RandFlip,RandZoom,Compose,Activations, AsDiscrete,EnsureChannelFirst
from monai.networks.nets import DenseNet121
import torchvision.transforms as transforms
import pydicom
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split


In [2]:
import neptune
from neptune.types import File
from neptune.utils import stringify_unsupported

In [3]:
# classe che uso come DatasetLoader
class BrainClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


Inizio con inizializzare Neptune e creare una nuova instanza di Neptune run

In [4]:
run = neptune.init_run(project='marioverd/brain-classification',
                       api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5MjRhMWVmZC0xMzhiLTQ4YzAtYjE4ZS0zNTlkOGJjOTkzNWEifQ==" )

https://app.neptune.ai/marioverd/brain-classification/e/BRAIN-5


### Log Hyperparameters

In [5]:
parameters = {
    "lr": 1e-5,
    "bs" : 20 ,
    "n_classes": 2,
    "model_filename": "densenet",
    "spatial_dimension": 2,
    "in_channels" : 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [6]:
from neptune.utils import stringify_unsupported

run["config/hyperparameters"] = stringify_unsupported(parameters)

### Log Config

Model and Dataset

In [7]:
model = DenseNet121(spatial_dims=parameters["spatial_dimension"], in_channels=parameters["in_channels"], out_channels=parameters["n_classes"]).to(parameters["device"])
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"])

In [11]:
run["config/model"] = type(model).__name__
run["config/criterion"] = type(criterion).__name__
run["config/optimizer"] = type(optimizer).__name__

In [8]:
data_dir = "./dataset/dcm/"
data_tfms = {
    "train": transforms.Compose(
        [
        LoadImage(image_only=True,ensure_channel_first= True),
        Resize((256,256)),
        NormalizeIntensity(),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
     ]
    ),
    "val": transforms.Compose(
        [
            LoadImage(image_only=True),
            EnsureChannelFirst() ,
            NormalizeIntensity(),
            Resize((256,256))
        ]
    ),
}

In [10]:
set_determinism(seed = 1234)
class_names = ["false","true"]   #dove true sono brain e false no brain 
num_class = len(class_names)
image_files = [
    [os.path.join(data_dir,class_names[i],x) for x in os.listdir(os.path.join(data_dir,class_names[i])) if x.endswith("dcm")] 
    for i in range(len(class_names))
    ]   #lista di liste, 0 contiene tutti i file labellati con no-brain.1 i brain

X = []
Y = []


for i in range(len(class_names)) :
    X.extend(image_files[i])
    Y.extend([i]*len(image_files[i]))


image_dictionary = {
                        "image" : X,
                        "label" : Y
}


In [11]:
train_x, test_x, train_y, test_y  = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
train_x, val_x, train_y, val_y  = train_test_split(train_x, train_y, test_size=0.2, stratify=train_y , random_state=42)
print("Training count : {} , Validation count : {} ,  Test count : {} ".format(len(train_x), len(val_x), len(test_x)))

Training count : 73 , Validation count : 19 ,  Test count : 24 


In [12]:
train_ds = BrainClassificationDataset(train_x, train_y, data_tfms["train"])
train_loader = DataLoader(train_ds, batch_size=20, shuffle=True,num_workers=5)

val_ds = BrainClassificationDataset(val_x, val_y, data_tfms["val"])
val_loader = DataLoader(val_ds, batch_size=20,num_workers=5)

test_ds = BrainClassificationDataset(test_x, test_y, data_tfms["val"])
test_loader = DataLoader(test_ds, batch_size=20,num_workers=5)
dataset_size = {"train": len(train_ds), "val": len(val_ds)}

In [13]:
run["config/dataset/path"] = data_dir
run["config/dataset/transforms"] = stringify_unsupported(data_tfms)
run["config/dataset/size"] = dataset_size

### Train Loop

In [14]:
for i, (x, y) in enumerate(train_loader, 0):
    x, y = x.to(parameters["device"]), y.to(parameters["device"])
    optimizer.zero_grad()
    outputs = model.forward(x)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, y)
    acc = (torch.sum(preds == y.data)) / len(x)

    run["training/batch/loss"].append(loss)

    run["training/batch/acc"].append(acc)

    loss.backward()
    optimizer.step()

2023-03-06 14:53:57,345 - Error while processing tag 0051100E


In [16]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marioverd/brain-classification/e/BRAIN-5/metadata
